In [36]:
import numpy as np
import pandas as pd
import re
import jieba
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfTransformer
from sklearn.feature_extraction.text import TfidfVectorizer, CountVectorizer
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import *
from sklearn.svm import SVC
from sklearn.model_selection import GridSearchCV
import xgboost as xgb
from sklearn.naive_bayes import MultinomialNB

In [4]:
df = pd.read_csv('random_3000_with_label.csv')

In [5]:
df.head()

,Index,user_id,created_at,content,like_num,repost_num,comment_num,manually_label,sentiment_baseline,senti_by_refined_snownlp,senti_by_jieba_sentiment
0,10289,2670985301,2020-02-08 23:52,抗击疫情 我们誓死不退（祝洛东） #武汉加油##抗击新型肺炎我们在行动##抗疫行动##万众...,6,0,0,1,1,1,1
1,33252,5622277724,2020-03-06 23:22,#甘肃新增17例境外输入新冠肺炎# 又来[费解]晕死真的是,1,0,1,0,-1,-1,1
2,15869,5936449524,2020-02-15 23:54,#新冠肺炎已开始在日本流行# 我靠,0,0,0,0,0,0,1
3,14422,2211096821,2020-02-13 23:59,日本再确诊1例新冠肺炎病例 总确诊数已达251例 http://t.cn/A6hGfqoZ,0,0,0,0,-1,-1,1
4,23309,6303064815,2020-02-23 23:59,//@微博辟谣:#微博辟谣# 金银潭医院：军运会五外籍运动员患的都是疟疾，与新冠肺炎无关,0,0,0,0,1,-1,1


In [6]:
df.drop(['sentiment_baseline','senti_by_refined_snownlp','senti_by_jieba_sentiment'],axis=1,inplace=True)
df.head()

,Index,user_id,created_at,content,like_num,repost_num,comment_num,manually_label
0,10289,2670985301,2020-02-08 23:52,抗击疫情 我们誓死不退（祝洛东） #武汉加油##抗击新型肺炎我们在行动##抗疫行动##万众...,6,0,0,1
1,33252,5622277724,2020-03-06 23:22,#甘肃新增17例境外输入新冠肺炎# 又来[费解]晕死真的是,1,0,1,0
2,15869,5936449524,2020-02-15 23:54,#新冠肺炎已开始在日本流行# 我靠,0,0,0,0
3,14422,2211096821,2020-02-13 23:59,日本再确诊1例新冠肺炎病例 总确诊数已达251例 http://t.cn/A6hGfqoZ,0,0,0,0
4,23309,6303064815,2020-02-23 23:59,//@微博辟谣:#微博辟谣# 金银潭医院：军运会五外籍运动员患的都是疟疾，与新冠肺炎无关,0,0,0,0


In [8]:
content_list = []
for comm in df.content.tolist():
    text = re.sub(r'(http[s]?://(?:[a-zA-Z]|[0-9]|[$-_@.&+]|[!*\(\),]|(?:%[0-9a-fA-F][0-9a-fA-F]))+)','',comm)
    text = re.sub(r'(?:回复)?(?://)?@[\w\u2E80-\u9FFF]+:?|\[\w+\]', ',',text)
    r='[’！？：；【】，、《》!"#$%&\'()（）“”…*+,-./:;<=>?@[\\]^_`{|}~]+'
    text = re.sub(r, '', text)
    content_list.append(text)

In [9]:
df['content'] = content_list

In [11]:
import jieba
df['content'] = df['content'].apply(lambda i:jieba.cut(i) )
df['content'] =[' '.join(i) for i in df['content']]
df.head()

Building prefix dict from the default dictionary ...
Loading model from cache C:\Users\asus\AppData\Local\Temp\jieba.cache
Loading model cost 0.761 seconds.
Prefix dict has been built successfully.


,Index,user_id,created_at,content,like_num,repost_num,comment_num,manually_label
0,10289,2670985301,2020-02-08 23:52,抗击 疫情 我们 誓死 不退 祝 洛东 武汉 加油 抗击 新型 肺炎 我们 在 ...,6,0,0,1
1,33252,5622277724,2020-03-06 23:22,甘肃 新增 17 例 境外 输入 新冠 肺炎 又 来 晕死 真的 是,1,0,1,0
2,15869,5936449524,2020-02-15 23:54,新冠 肺炎 已 开始 在 日本 流行 我 靠,0,0,0,0
3,14422,2211096821,2020-02-13 23:59,日本 再 确诊 1 例新冠 肺炎 病例 总 确诊 数已 达 251 例,0,0,0,0
4,23309,6303064815,2020-02-23 23:59,微博 辟谣 金银 潭 医院 军运会 五 外籍 运动员 患 的 都 是 疟疾 与 新冠 肺...,0,0,0,0


In [14]:
y = df.manually_label.values
x_train, x_valid, y_train, y_valid = train_test_split(df.content.values, y, 
                                                  stratify=y, 
                                                  random_state=42, 
                                                  test_size=0.2, shuffle=True)
print (x_train.shape)
print (x_valid.shape)

(2400,)
(600,)


In [16]:
def multiclass_logloss(actual, predicted, eps=1e-15):
    """对数损失度量（Logarithmic Loss  Metric）的多分类版本。
    :param actual: 包含actual target classes的数组
    :param predicted: 分类预测结果矩阵, 每个类别都有一个概率
    """
    # Convert 'actual' to a binary array if it's not already:
    if len(actual.shape) == 1:
        actual2 = np.zeros((actual.shape[0], predicted.shape[1]))
        for i, val in enumerate(actual):
            actual2[i, val] = 1
        actual = actual2

    clip = np.clip(predicted, eps, 1 - eps)
    rows = actual.shape[0]
    vsota = np.sum(actual * np.log(clip))
    return -1.0 / rows * vsota

In [26]:
def number_normalizer(tokens):
    """ 将所有数字标记映射为一个占位符（Placeholder）。
    对于许多实际应用场景来说，以数字开头的tokens不是很有用，
    但这样tokens的存在也有一定相关性。 通过将所有数字都表示成同一个符号，可以达到降维的目的。
    """
    return ("#NUMBER" if token[0].isdigit() else token for token in tokens)


class NumberNormalizingVectorizer(TfidfVectorizer):
    def build_tokenizer(self):
        tokenize = super(NumberNormalizingVectorizer, self).build_tokenizer()
        return lambda doc: list(number_normalizer(tokenize(doc)))

stwlist=[line.strip() for line in open('cn_stopwords.txt',
'r',encoding='utf-8').readlines()]
tfv = NumberNormalizingVectorizer(min_df=3,  
                                  max_df=0.5,
                                  max_features=None,                 
                                  ngram_range=(1,2), 
                                  use_idf=True,
                                  smooth_idf=True,
                                  stop_words = stwlist)

tfv.fit(list(x_train) + list(x_valid))
x_train_tfv =  tfv.transform(x_train) 
x_valid_tfv = tfv.transform(x_valid)

tfv.fit(list(x_train) + list(x_valid))
x_train_tfv =  tfv.transform(x_train) 
x_valid_tfv = tfv.transform(x_valid)

#### Bayes

In [37]:
clf = MultinomialNB()
clf.fit(x_train_tfv, y_train)
y_pred_proba = clf.predict_proba(x_valid_tfv)
y_pred = clf.predict(x_valid_tfv)
print ("logloss: %0.3f " % multiclass_logloss(y_valid, y_pred_proba))
print(classification_report(y_valid, y_pred,target_names = ['-1','0','1']))

logloss: 1.765 
              precision    recall  f1-score   support

          -1       0.72      0.43      0.54       161
           0       0.62      0.85      0.72       258
           1       0.73      0.60      0.66       181

    accuracy                           0.66       600
   macro avg       0.69      0.63      0.64       600
weighted avg       0.68      0.66      0.65       600



#### Logistic Regression

In [27]:
clf = LogisticRegression(C=1.0,solver='lbfgs',multi_class='multinomial')
clf.fit(x_train_tfv, y_train)
y_pred_proba = clf.predict_proba(x_valid_tfv)
y_pred = clf.predict(x_valid_tfv)
print ("logloss: %0.3f " % multiclass_logloss(y_valid, y_pred_proba))
print(classification_report(y_valid, y_pred,target_names = ['-1','0','1']))

logloss: 1.633 
              precision    recall  f1-score   support

          -1       0.64      0.53      0.58       161
           0       0.64      0.79      0.71       258
           1       0.76      0.61      0.68       181

    accuracy                           0.67       600
   macro avg       0.68      0.65      0.66       600
weighted avg       0.68      0.67      0.67       600



#### SVM

In [31]:
%%time
clf = SVC(C=1.0,kernel='rbf',probability=True)
clf.fit(x_train_tfv, y_train)
y_pred_proba = clf.predict_proba(x_valid_tfv)
y_pred = clf.predict(x_valid_tfv)
print ("logloss: %0.3f " % multiclass_logloss(y_valid, y_pred_proba))
print(classification_report(y_valid, y_pred,target_names = ['-1','0','1']))

logloss: 1.926 
              precision    recall  f1-score   support

          -1       0.60      0.47      0.52       161
           0       0.60      0.79      0.68       258
           1       0.74      0.56      0.64       181

    accuracy                           0.63       600
   macro avg       0.65      0.61      0.62       600
weighted avg       0.64      0.63      0.63       600

Wall time: 8.63 s


#### XGBoost

In [34]:
clf = xgb.XGBClassifier(max_depth=7, n_estimators=200, colsample_bytree=0.8, 
                        subsample=0.8, nthread=10, learning_rate=0.1)
clf.fit(x_train_tfv, y_train)
y_pred_proba = clf.predict_proba(x_valid_tfv)
y_pred = clf.predict(x_valid_tfv)
print ("logloss: %0.3f " % multiclass_logloss(y_valid, y_pred_proba))
print(classification_report(y_valid, y_pred,target_names = ['-1','0','1']))

logloss: 2.097 
              precision    recall  f1-score   support

          -1       0.60      0.55      0.57       161
           0       0.65      0.75      0.70       258
           1       0.73      0.62      0.67       181

    accuracy                           0.66       600
   macro avg       0.66      0.64      0.65       600
weighted avg       0.66      0.66      0.65       600

